In [71]:
import pandas as pd
import numpy as np

# IMU Changes
- IMU data was recorded at 200hz.
- Because of the high frequency of this data, adding deltas (between 1000 and 1100 microseconds) were not significant to cause Kimera-VIO to crash.
- Will now modify raw data and change it to 100 hz.

In [72]:
df = pd.read_csv('./mav0/imu0/data.csv')

In [73]:
df['#timestamp [ns]'] = df['#timestamp [ns]'].apply(lambda x: f"{x:.0f}")
df.head(5)

,#timestamp [ns],w_RS_S_x [rad s^-1],w_RS_S_y [rad s^-1],w_RS_S_z [rad s^-1],a_RS_S_x [m s^-2],a_RS_S_y [m s^-2],a_RS_S_z [m s^-2]
0,1413393212225760512,0.094946,0.020246,0.058643,9.128357,0.106239,-2.606934
1,1413393212230760448,0.058643,0.035605,0.051662,9.422556,1.160454,-2.623279
2,1413393212235760384,-0.011170,0.027925,0.069813,9.398040,1.618097,-2.705001
3,1413393212240760576,-0.072606,0.033510,0.069813,9.496106,0.670121,-2.819412
4,1413393212245760512,-0.092153,0.036303,0.078889,8.989429,-0.130755,-2.713173


In [74]:
df['#timestamp [ns]'] = df['#timestamp [ns]'].astype('float64')

In [75]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22800 entries, 0 to 22799
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   #timestamp [ns]      22800 non-null  float64
 1   w_RS_S_x [rad s^-1]  22800 non-null  float64
 2   w_RS_S_y [rad s^-1]  22800 non-null  float64
 3   w_RS_S_z [rad s^-1]  22800 non-null  float64
 4   a_RS_S_x [m s^-2]    22800 non-null  float64
 5   a_RS_S_y [m s^-2]    22800 non-null  float64
 6   a_RS_S_z [m s^-2]    22800 non-null  float64
dtypes: float64(7)
memory usage: 1.2 MB


In [76]:
# Check the deltas.
orig_deltas = np.unique(np.diff(df['#timestamp [ns]']), return_counts=True)

In [77]:
original_rate = 200
reduction_factor = 1
print("The frequency will be reduced to:", original_rate/reduction_factor)

The frequency will be reduced to: 200.0


In [78]:
# Reducing frequency
df = df.iloc[::reduction_factor]

In [79]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22800 entries, 0 to 22799
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   #timestamp [ns]      22800 non-null  float64
 1   w_RS_S_x [rad s^-1]  22800 non-null  float64
 2   w_RS_S_y [rad s^-1]  22800 non-null  float64
 3   w_RS_S_z [rad s^-1]  22800 non-null  float64
 4   a_RS_S_x [m s^-2]    22800 non-null  float64
 5   a_RS_S_y [m s^-2]    22800 non-null  float64
 6   a_RS_S_z [m s^-2]    22800 non-null  float64
dtypes: float64(7)
memory usage: 1.2 MB


In [80]:
# Check deltas after downsampling.
new_deltas = np.unique(np.diff(df['#timestamp [ns]']), return_counts=True)

In [81]:
#Since we have reduced the frequency, we should get a fractional value when we divide the original deltas by the new deltas
orig_deltas[0] / new_deltas[0]

array([1., 1.])

In [82]:
df.head(10)

,#timestamp [ns],w_RS_S_x [rad s^-1],w_RS_S_y [rad s^-1],w_RS_S_z [rad s^-1],a_RS_S_x [m s^-2],a_RS_S_y [m s^-2],a_RS_S_z [m s^-2]
0,1.413393e+18,0.094946,0.020246,0.058643,9.128357,0.106239,-2.606934
1,1.413393e+18,0.058643,0.035605,0.051662,9.422556,1.160454,-2.623279
2,1.413393e+18,-0.011170,0.027925,0.069813,9.398040,1.618097,-2.705001
3,1.413393e+18,-0.072606,0.033510,0.069813,9.496106,0.670121,-2.819412
4,1.413393e+18,-0.092153,0.036303,0.078889,8.989429,-0.130755,-2.713173
5,1.413393e+18,-0.058643,0.024435,0.100531,9.177390,-0.890771,-2.582418
6,1.413393e+18,-0.005585,0.009076,0.113795,9.299973,-1.234003,-2.476179
7,1.413393e+18,0.037001,0.013265,0.100531,9.586000,-0.817221,-2.745862
8,1.413393e+18,0.061436,0.023736,0.091455,9.275456,0.073550,-2.582418
9,1.413393e+18,0.044680,0.017453,0.076794,9.536967,0.498505,-2.688657


In [83]:
max_nano_seconds_to_add = max(np.unique(np.diff(df['#timestamp [ns]'])))

In [84]:
max_nano_seconds_to_add

np.float64(5000192.0)

In [85]:
(max_nano_seconds_to_add - 1)/2

np.float64(2500095.5)

### Introducing Minor Variation in IMU Timestamps

- 1,000,000 ns = 1 ms
- 10,000,000 ns = 10 ms (current delta)
- We are introducing at most 5,000,000 ns (5ms) jitter.
- In a worst case scenario, we can have 5,00,000 ns being subtracted and then for the next data point, we we can subtract none, leaving a max delta of 15,000,000 ns = 15 ms.
- This means that in terms of 'jitter', we are introducing +/- 5ms.
- If we introduce jitter of more than 10ms, we will have instances where timestamps overlap!

In [86]:
random_nanoseconds_to_add = np.random.uniform(low=0, high=(max_nano_seconds_to_add - 1)/2, size=10)

In [87]:
random_nanoseconds_to_add

array([1794120.97315939,  542410.29728864, 1676581.2835025 ,
       2498557.958745  , 1188593.52587177,  693381.75737306,
        833362.97977501,  335888.82373984, 2283520.75735962,
       1303297.28384819])

In [88]:
default_ts_arr = df['#timestamp [ns]'].values

In [89]:
np.unique(np.diff(default_ts_arr))

array([4999936., 5000192.])

In [90]:
modified_ts_arr = np.array([])

In [91]:
for def_ts in default_ts_arr:
    modified_ts = float(def_ts - np.random.choice(random_nanoseconds_to_add, 1)[0])
    modified_ts_arr = np.append(modified_ts_arr, modified_ts)

In [92]:
np.unique(np.diff(modified_ts_arr))

array([2837248., 2837504., 3043840., 3044096., 3052288., 3052544.,
       3194880., 3195136., 3258880., 3259136., 3334656., 3334912.,
       3409920., 3410176., 3541760., 3542016., 3549696., 3549952.,
       3659264., 3659520., 3689984., 3690240., 3748352., 3748608.,
       3804672., 3804928., 3865856., 3866112., 3899392., 3899648.,
       3905024., 3905280., 4016896., 4017152., 4019712., 4019968.,
       4032512., 4032768., 4039168., 4039424., 4147200., 4147456.,
       4156672., 4156928., 4177920., 4178176., 4239104., 4239360.,
       4295424., 4295680., 4353792., 4354048., 4390144., 4390400.,
       4392960., 4393216., 4394496., 4394752., 4502528., 4502784.,
       4504832., 4505088., 4509184., 4509440., 4510464., 4510720.,
       4512000., 4512256., 4529920., 4530176., 4626688., 4626944.,
       4642304., 4642560., 4644608., 4644864., 4709120., 4709376.,
       4784896., 4785152., 4793344., 4793600., 4848896., 4849152.,
       4860160., 4860416., 4882432., 4882688., 4885248., 48855

In [93]:
np.unique(np.diff(modified_ts_arr)) > max_nano_seconds_to_add * 1.5

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [94]:
df['#timestamp [ns]'] = modified_ts_arr

In [95]:
np.unique(np.diff(df['#timestamp [ns]']), return_counts=True)

(array([2837248., 2837504., 3043840., 3044096., 3052288., 3052544.,
        3194880., 3195136., 3258880., 3259136., 3334656., 3334912.,
        3409920., 3410176., 3541760., 3542016., 3549696., 3549952.,
        3659264., 3659520., 3689984., 3690240., 3748352., 3748608.,
        3804672., 3804928., 3865856., 3866112., 3899392., 3899648.,
        3905024., 3905280., 4016896., 4017152., 4019712., 4019968.,
        4032512., 4032768., 4039168., 4039424., 4147200., 4147456.,
        4156672., 4156928., 4177920., 4178176., 4239104., 4239360.,
        4295424., 4295680., 4353792., 4354048., 4390144., 4390400.,
        4392960., 4393216., 4394496., 4394752., 4502528., 4502784.,
        4504832., 4505088., 4509184., 4509440., 4510464., 4510720.,
        4512000., 4512256., 4529920., 4530176., 4626688., 4626944.,
        4642304., 4642560., 4644608., 4644864., 4709120., 4709376.,
        4784896., 4785152., 4793344., 4793600., 4848896., 4849152.,
        4860160., 4860416., 4882432., 4882688., 

### End Section

In [96]:
df.head(10)

,#timestamp [ns],w_RS_S_x [rad s^-1],w_RS_S_y [rad s^-1],w_RS_S_z [rad s^-1],a_RS_S_x [m s^-2],a_RS_S_y [m s^-2],a_RS_S_z [m s^-2]
0,1.413393e+18,0.094946,0.020246,0.058643,9.128357,0.106239,-2.606934
1,1.413393e+18,0.058643,0.035605,0.051662,9.422556,1.160454,-2.623279
2,1.413393e+18,-0.011170,0.027925,0.069813,9.398040,1.618097,-2.705001
3,1.413393e+18,-0.072606,0.033510,0.069813,9.496106,0.670121,-2.819412
4,1.413393e+18,-0.092153,0.036303,0.078889,8.989429,-0.130755,-2.713173
5,1.413393e+18,-0.058643,0.024435,0.100531,9.177390,-0.890771,-2.582418
6,1.413393e+18,-0.005585,0.009076,0.113795,9.299973,-1.234003,-2.476179
7,1.413393e+18,0.037001,0.013265,0.100531,9.586000,-0.817221,-2.745862
8,1.413393e+18,0.061436,0.023736,0.091455,9.275456,0.073550,-2.582418
9,1.413393e+18,0.044680,0.017453,0.076794,9.536967,0.498505,-2.688657


In [97]:
df['#timestamp [ns]'] = df['#timestamp [ns]'].apply(lambda x: f"{x:.0f}")

In [98]:
df.head(10)

,#timestamp [ns],w_RS_S_x [rad s^-1],w_RS_S_y [rad s^-1],w_RS_S_z [rad s^-1],a_RS_S_x [m s^-2],a_RS_S_y [m s^-2],a_RS_S_z [m s^-2]
0,1413393212224457216,0.094946,0.020246,0.058643,9.128357,0.106239,-2.606934
1,1413393212229457152,0.058643,0.035605,0.051662,9.422556,1.160454,-2.623279
2,1413393212233966336,-0.011170,0.027925,0.069813,9.398040,1.618097,-2.705001
3,1413393212239084032,-0.072606,0.033510,0.069813,9.496106,0.670121,-2.819412
4,1413393212243476992,-0.092153,0.036303,0.078889,8.989429,-0.130755,-2.713173
5,1413393212249571840,-0.058643,0.024435,0.100531,9.177390,-0.890771,-2.582418
6,1413393212254571776,-0.005585,0.009076,0.113795,9.299973,-1.234003,-2.476179
7,1413393212258966528,0.037001,0.013265,0.100531,9.586000,-0.817221,-2.745862
8,1413393212264571904,0.061436,0.023736,0.091455,9.275456,0.073550,-2.582418
9,1413393212269927168,0.044680,0.017453,0.076794,9.536967,0.498505,-2.688657


In [99]:
df.to_csv('./mav0/imu0/data_modified.csv', index=False)

In [100]:
!cp ./mav0/imu0/data_modified.csv /home/shiva/Datasets/V2_01_easy/V2_01_easy/mav0/imu0/data.csv